---
# Name of Method - Model 4
---
In this notebook, ...


## Results

---

### Importing necessary library

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay

from features import features

from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

### Read data file

In [2]:
churn_df = pd.read_excel('../data/churn_cleaned_featEng.xlsx')
churn_df

,Latitude,Longitude,Tenure Months,Monthly Charges,Churn Value,Senior Citizen_Yes,Partner_Yes,Dependents_Yes,Internet Service_Fiber optic,Internet Service_No,...,Payment Method_Electronic check,Payment Method_Mailed check,Gender_Male,Phone Service_Yes,Multiple Lines_No phone service,Multiple Lines_Yes,Streaming TV_No internet service,Streaming TV_Yes,Streaming Movies_No internet service,Streaming Movies_Yes
0,33.964131,118.272783,2,53.85,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
1,34.059281,118.307420,2,70.70,1,0,0,1,1,0,...,1,0,0,1,0,0,0,0,0,0
2,34.048013,118.293953,8,99.65,1,0,0,1,1,0,...,1,0,0,1,0,1,0,1,0,1
3,34.062125,118.315709,28,104.80,1,0,1,1,1,0,...,1,0,0,1,0,1,0,1,0,1
4,34.039224,118.266293,49,103.70,1,0,0,1,1,0,...,0,0,1,1,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,34.341737,116.539416,72,21.15,0,0,0,0,0,1,...,0,0,0,1,0,0,1,0,1,0
7039,34.667815,117.536183,24,84.80,0,0,1,1,0,0,...,0,1,1,1,0,1,0,1,0,1
7040,34.559882,115.637164,72,103.20,0,0,1,1,1,0,...,0,0,0,1,0,1,0,1,0,1
7041,34.167800,116.864330,11,29.60,0,0,1,1,0,0,...,1,0,0,0,1,0,0,0,0,0


---

<center>
    
## Preparing data

</center>

---

### Separate X and y features

In [3]:
# Seperate X and y features
# Contract_Two year	4.239139e-01
# 1	Dependents_Yes	1.919674e-01
# 2	Contract_One year	1.837644e-01
# 3	Streaming Movies_No internet service	1.135717e-01
feat_list = [ 'Dependents_Yes', 'Contract_Two year','Contract_One year', 'Internet Service_No',
              'Internet Service_Fiber optic']
#['Tenure Months', 'Dependents_Yes', 'Internet Service_Fiber optic','Payment Method_Electronic check','Contract_Two year','Contract_One year']
X = churn_df.drop(columns=['Churn Value'])#[feat_list]
y = churn_df['Churn Value']

### Split dataset (training/testing)

In [4]:
# Separating the dataset into a training dataset (70%) and testing+validation (30%) dataset
X_train, X_test_validation, y_train, y_test_validation = train_test_split(X, y, train_size=0.7, random_state=5)

# Separating the testing+valisation dataset into a testing dataset (15%) and a validation dataset (15%) 
X_val, X_test, y_val, y_test = train_test_split(X_test_validation, y_test_validation, test_size=0.5, random_state=5)

### Converting data subset to dataframe 

In [5]:
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

### Scale X features

In [6]:
# Create instance of scaler
scaler = StandardScaler()

# Scale the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

---

<center>
    
## Decision Tree Model

</center>

---

### Find best parameters for model
---

**Create intance of model**

In [7]:
dtc = DecisionTreeClassifier(random_state=25) #'balanced'

In [8]:
 # pipeline = ImbPipeline([
 #     ('smote', SMOTE(random_state=42)),
 #     ('dtc', DecisionTreeClassifier(random_state=42))
 # ])

**Setting parameters for the GridSearch**

In [9]:
param_grid = { 'criterion' : ['gini', 'entropy', 'log_loss'],
               'splitter' : ['best','random'],
               'max_depth' : [3,4,5,6,7,8,9,10],
               'class_weight': ['balanced', {0: 1, 1: 1}, {0: 0.7, 1: 1.3}, {0: 0.5, 1: 1.5}, {0: 0.3, 1: 1.7}, {0: 0.1, 1: 1.9}],
               'max_features' :['sqrt', 'log2', None]
             }

# param_grid = { 'dtc__criterion' : ['gini', 'entropy', 'log_loss'],
#                 'dtc__splitter' : ['best','random'],
#                 'dtc__max_depth' : [3,4,5,6,7,8,9,10],
#                 'dtc__class_weight': ['balanced', {0: 1, 1: 1}, {0: 0.7, 1: 1.3}, {0: 0.5, 1: 1.5}, {0: 0.3, 1: 1.7}, {0: 0.1, 1: 1.9}],
#                 'dtc__max_features' :['sqrt', 'log2', None]
#              }
# param_grid = {
#     'clf__max_depth': [3, 5, 10],
#     'clf__min_samples_split': [2, 5, 10],
#     'threshold': [0.1, 0.2, 0.3, 0.4, 0.5]  # Threshold Grid
# }
scoring = ['balanced_accuracy','recall', 'precision']
#scoring = ['balanced_accuracy', 'f1_macro', 'roc_auc','recall']
refit = 'balanced_accuracy'
#refit = 'f1_weighted' #'roc_auc'

**Setting instance of GridSearchCV**

In [10]:
grid = GridSearchCV(dtc, param_grid, scoring=scoring, refit=refit)

**Train the models to find best parameters**

In [11]:
np.unique(y)

array([0, 1])

In [12]:
grid.fit(X_train_scaled,y_train)

/home/abf/Dev/graph/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/abf/Dev/graph/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/abf/Dev/graph/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/abf/Dev/graph/lib/python3.12/site-packag

GridSearchCV(estimator=DecisionTreeClassifier(random_state=25),
             param_grid={'class_weight': ['balanced', {0: 1, 1: 1},
                                          {0: 0.7, 1: 1.3}, {0: 0.5, 1: 1.5},
                                          {0: 0.3, 1: 1.7}, {0: 0.1, 1: 1.9}],
                         'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
                         'max_features': ['sqrt', 'log2', None],
                         'splitter': ['best', 'random']},
             refit='balanced_accuracy',
             scoring=['balanced_accuracy', 'recall', 'precision'])

---
### Investigate best model's predictive features

---

In [13]:
grid.best_params_

{'class_weight': {0: 0.5, 1: 1.5},
 'criterion': 'gini',
 'max_depth': 6,
 'max_features': None,
 'splitter': 'best'}

In [14]:
grid.best_score_

np.float64(0.7710860291337124)

In [15]:
grid.n_features_in_

27

In [16]:
model = grid.best_estimator_

In [17]:
best_model = grid.best_estimator_

# Get feature importances from the best model
feature_importance = best_model.feature_importances_
#feature_importance = best_model.named_steps['dtc'].feature_importances_

feature_importance

array([0.03256595, 0.01640727, 0.0991747 , 0.02888883, 0.00368015,
       0.        , 0.09356955, 0.09280828, 0.        , 0.00389108,
       0.        , 0.        , 0.        , 0.20909243, 0.35448574,
       0.        , 0.        , 0.00619166, 0.        , 0.        ,
       0.        , 0.01203338, 0.        , 0.        , 0.        ,
       0.01344037, 0.03377062])

In [18]:
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_df

,Feature,Importance
0,Contract_Two year,0.354486
1,Contract_One year,0.209092
2,Tenure Months,0.099175
3,Dependents_Yes,0.093570
4,Internet Service_Fiber optic,0.092808
5,Streaming Movies_Yes,0.033771
6,Latitude,0.032566
7,Monthly Charges,0.028889
8,Longitude,0.016407
9,Streaming Movies_No internet service,0.013440


In [19]:
grid.best_estimator_.predict_proba(X_train_scaled)

array([[0.25070956, 0.74929044],
       [0.25070956, 0.74929044],
       [0.2625    , 0.7375    ],
       ...,
       [1.        , 0.        ],
       [0.59139785, 0.40860215],
       [1.        , 0.        ]], shape=(4930, 2))

**Plot Decision Tree Arborescence**

In [ ]:
# Create figure and adjust size
plt.figure(figsize=(100,100))

# Plot decision tree
plot_tree(model, feature_names=X_train.columns)

# Save decision tree
plt.savefig('../graph/DecisionTreeClassifier.png')

# Show model
plt.show()

---
### Get scores for the training dataset
---

**Make prediction on training dataset**

In [ ]:
y_pred_train = grid.predict(X_train_scaled)

**Create Confusion Matrix**

In [ ]:
cm = confusion_matrix(y_train, y_pred_train)
cm

**Plot confusion matrix**

In [ ]:
# Create figure and adjsut its size
plt.figure(figsize=(4,4))

# Create plot for Confusion Matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=grid.classes_)

disp.plot()

# Show plot
plt.show()

**Print classification Report**

In [ ]:
print(classification_report(y_train,y_pred_train))

---

<center>
    
## Validation

</center>

---

**Run model on testing dataset**

In [ ]:
y_pred_test = grid.predict(X_test_scaled)

**Create Confusion Matrix**

In [ ]:
cm = confusion_matrix(y_test, y_pred_test)
cm

**Plot confusion matrix**

In [ ]:
# Create figure and adjsut its size
plt.figure(figsize=(4,4))

# Create plot for Confusion Matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=grid.classes_)

disp.plot()

# Show plot
plt.show()

**Print classification Report**

In [ ]:
print(classification_report(y_test,y_pred_test))

In [ ]:
y_pred_prob = grid.predict_proba(X_test_scaled)[:, 1]  # Probabilities for class 1

# Set a custom threshold (e.g., 0.6 instead of 0.5)
threshold = .7
y_pred_custom = (y_pred_prob >= threshold).astype(int)

# Evaluate the classification performance
print("Classification Report at Threshold 0.6:")
print(classification_report(y_test, y_pred_custom))

---

<center>
    
## Results

</center>

---

### Results

all = 'Senior Citizen', 'Partner', 'Dependents', 'Tenure Months', 
            'Internet Service', 
            'Online Security', 'Online Backup', 'Device Protection',
            'Tech Support', 'Contract', 'Paperless Billing', 'Payment Method',
            'Monthly Charges', 'Churn Value',

Training dataset (all+ lat/lon)
| Metric       | Precision | Recall | F1-Score | Support |
|-------------|-----------|-------|----------|--------|
| Class 0     | 0.90     | 0.79  | 0.84    | 4132   |
| Class 1     | 0.57     | 0.76  | 0.65    | 1502   |
| Accuracy    |          |       | 0.78    | 5634   |
| Macro Avg   | 0.73     | 0.78  | 0.74    | 5634   |
| Weighted Avg| 0.81     | 0.78  | 0.79    | 5634   |

test(all + lat/lon)
| Metric       | Precision | Recall | F1-Score | Support |
|-------------|-----------|-------|----------|--------|
| Class 0     | 0.90     | 0.75  | 0.81    | 504    |
| Class 1     | 0.55     | 0.78  | 0.65    | 201    |
| Accuracy    |          |       | 0.76    | 705    |
| Macro Avg   | 0.72     | 0.76  | 0.73    | 705    |
| Weighted Avg| 0.80     | 0.76  | 0.77    | 705    |